In [ ]:
# Necessary Imports
import cv2
import pathlib
import numpy as np
import pandas as pd
import tensorflow as tf
import os
# View an image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random
from PIL import Image

In [ ]:
# Path to Kaggle Input
path = "../input/indian-monuments-image-dataset/Indian-monuments/images"
# Walk through the directory and list number of files
for dirpath, dirnames, filenames in os.walk(path):
  print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

In [ ]:
train_dir =  path + "/train/"
test_dir = path + "/test/"
train_dir

In [ ]:
# get all the class names
data_dir = pathlib.Path(train_dir)
class_names = np.array(sorted([item.name for item in data_dir.glob("*")]))
class_names

In [ ]:
def plot_loss_curves(history):
  """
  Returns separate loss curves for training and validation metrics.
  """ 
  loss = history.history['loss']
  val_loss = history.history['val_loss']

  accuracy = history.history['accuracy']
  val_accuracy = history.history['val_accuracy']

  epochs = range(len(history.history['loss']))

  # Plot loss
  plt.plot(epochs, loss, label='training_loss')
  plt.plot(epochs, val_loss, label='val_loss')
  plt.title('Loss')
  plt.xlabel('Epochs')
  plt.legend()

  # Plot accuracy
  plt.figure()
  plt.plot(epochs, accuracy, label='training_accuracy')
  plt.plot(epochs, val_accuracy, label='val_accuracy')
  plt.title('Accuracy')
  plt.xlabel('Epochs')
  plt.legend();

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense

# Rescale the data and create data generator instances
train_datagen = ImageDataGenerator(rescale=1/255.)
test_datagen = ImageDataGenerator(rescale=1/255.)

# Load data in from directories and turn it into batches
train_data = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    validation_split=0.2,
    image_size=(224, 224),
    batch_size= 32,
    shuffle = True,
    subset='training',
    seed = 123,
    label_mode = 'categorical',
)

valid_data = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    validation_split=0.2,
    image_size=(224, 224),
    batch_size= 32,
    shuffle = False,
    subset='validation',
    seed = 123,
    label_mode = 'categorical',
)


test_data = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    image_size=(224, 224),
    batch_size= 32,
    shuffle = False,
    seed = 123,
    label_mode = 'categorical',
)

print(len(train_data))
print(len(valid_data))
print(len(test_data))

In [ ]:
# from keras.applications import DenseNet121
# from keras.preprocessing.image import ImageDataGenerator
# from keras.models import Sequential
# from keras.layers import Dense, GlobalAveragePooling2D,  Flatten
# import keras


# # Define the DenseNet model
# densenet = DenseNet121(weights='imagenet', include_top=False, input_shape=(224,224,3))


# # Add a global average pooling layer
# # x = train_data
# # preprocess_input = tf.keras.applications.densenet.preprocess_input
# # x = preprocess_input(x)
# x =  densenet.output
# x = Flatten()(x)
# # x = GlobalAveragePooling2D()(x)

# # Add a dense layer
# predictions = Dense(24, activation='softmax')(x)

# # Wrap the predictions layer in a KerasModel object
# predictions_model = keras.Model(inputs=x, outputs=predictions)


# # Define the model
# model = Sequential()
# model.add(densenet)
# model.add(predictions_model)

# # Compile the model
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# # Train the model
# model.fit(train_data, epochs=5, validation_data=test_data)


In [ ]:
bad_paths = []

for image_path in train_dir:
    try:
      img_bytes = tf.io.read_file(path)
      decoded_img = tf.io.decode_image(img_bytes)
    except tf.errors.InvalidArgumentError as e:
      print(f"Found bad path {image_path}...{e}")
      bad_paths.append(image_path)

    print(f"{image_path}: OK")

print("BAD PATHS:")
for bad_path in bad_paths:
    print(f"{bad_path}")

In [ ]:
from keras.applications import DenseNet121
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, GlobalAveragePooling2D, Flatten
import keras


model = tf.keras.Sequential([
        #resize_and_rescale,
        data_augmentation,
        tf.keras.applications.DenseNet121(
            input_shape=(224, 224, 3),
            weights='imagenet',
            include_top=False,
            classes=24,
        ),
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(24, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit( train_data, 
    validation_data=valid_data,
          epochs=10)


In [ ]:
train_data = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    validation_split=0.2,
    image_size=(224, 224),
    batch_size= 32,
    shuffle = True,
    subset='training',
    seed = 123,
    label_mode = 'categorical',
)

valid_data = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    validation_split=0.2,
    image_size=(224, 224),
    batch_size= 32,
    shuffle = False,
    subset='validation',
    seed = 123,
    label_mode = 'categorical',
)

print(len(train_data))
print(len(valid_data))


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers


resize_and_rescale = tf.keras.Sequential([
  layers.experimental.preprocessing.Resizing(224, 224),
  layers.experimental.preprocessing.Rescaling(1 /255)
])

# Data augmentation to reduce overtraining
data_augmentation = tf.keras.Sequential([
     layers.experimental.preprocessing.RandomFlip("horizontal", input_shape=(224, 224,3)),
    layers.experimental.preprocessing.RandomRotation(0.1),
    layers.experimental.preprocessing.RandomZoom(0.1),
])



model = tf.keras.Sequential([
        #resize_and_rescale,
        data_augmentation,
        tf.keras.applications.EfficientNetB0(
            input_shape=(224, 224, 3),
            weights='imagenet',
            include_top=False,
            drop_connect_rate=0.5
        ),
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(24, activation='softmax') 
])


In [ ]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics=['accuracy']) 

history = model.fit(
    train_data, 
    validation_data=valid_data,
    epochs= 20,
)